In [1]:
import nest_asyncio
import asyncio
from twikit import Client
import pandas as pd
import time
import random
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import fitz
import re
import locale
from datetime import datetime

In [2]:
#locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # o 'es_MX.UTF-8' en algunos sistemas

In [3]:
os.makedirs("gacetas_descargadas", exist_ok=True)
#Dentro de gacetas_descargadas genera otra carpeta que sea resultados_gacetas
os.makedirs("gacetas_descargadas/resultados_gacetas", exist_ok=True)

In [4]:
nest_asyncio.apply()

In [5]:
with open("cuentas_x_1.txt", "r", encoding="utf-8") as f:
    cuentas = [line.strip().split(",") for line in f]
#Número de cuentas
print(f"🔐 Número de cuentas: {len(cuentas)}")
USERNAME, EMAIL, PASSWORD = random.choice(cuentas)
print(f"🔐 Conectando con: {USERNAME}")

🔐 Número de cuentas: 1
🔐 Conectando con: IpdpCpyp


In [6]:
USERNAME='IpdpCpyp'
PASSWORD='Instituto2025'
EMAIL='ipdp.cpyp@gmail.com'

In [7]:
client = Client("es-ES")

In [8]:
async def login_and_search_multiple(queries, max_retries=3, retry_delay=30):
    await client.login(
        auth_info_1=USERNAME,
        auth_info_2=EMAIL,
        password=PASSWORD
    )

    all_data = []

    for query in queries:
        print(f"🔍 Buscando: {query}")
        retries = 0
        while retries <= max_retries:
            try:
                tweets = await client.search_tweet(query, "Latest")

                for tweet in tweets:
                    all_data.append({
                        "busqueda": query,
                        "usuario": tweet.user.name,
                        "usuario_id": tweet.user.id,
                        "texto": tweet.text,
                        "fecha": tweet.created_at,
                        "url": tweet.urls
                    })

                # Espera aleatoria entre búsquedas
                sleep_time = random.uniform(5, 12)
                print(f"⏳ Esperando {sleep_time:.1f} segundos...")
                await asyncio.sleep(sleep_time)
                break  # salir del ciclo si la búsqueda fue exitosa

            except Exception as e:
                print(f"⚠️ Error al buscar '{query}': {e}")
                if "Rate limit exceeded" in str(e).lower():
                    retries += 1
                    print(f"🔁 Reintentando en {retry_delay} segundos... (Intento {retries}/{max_retries})")
                    await asyncio.sleep(retry_delay)
                else:
                    break  # otro tipo de error, salir sin reintentar

    df = pd.DataFrame(all_data)
    return df


In [9]:
busquedas = ["gaceta oficial cdmx"]

In [10]:
df_resultados = await login_and_search_multiple(busquedas, max_retries=5, retry_delay=60)
#df_resultados['fecha'] = pd.to_datetime(df_resultados['fecha'], format="%a %b %d %H:%M:%S %z %Y", errors='coerce')

🔍 Buscando: gaceta oficial cdmx
⏳ Esperando 7.1 segundos...


In [11]:
consejeria = df_resultados[df_resultados['usuario'].str.contains("ConsejeríaCDMX", case=False, na=False)]
consejeria= consejeria[consejeria['texto'].str.contains("Consulta la Gaceta Oficial", case=False, na=False)]
consejeria['link_gaceta'] = consejeria['url'].apply(lambda x: x[0]['expanded_url'] if isinstance(x, list) and len(x) > 0 and 'expanded_url' in x[0] else None)
consejeria['fecha'] = pd.to_datetime(consejeria['fecha'], format="%a %b %d %H:%M:%S %z %Y", errors='coerce')
#Ordenar por fecha
consejeria= consejeria.sort_values('fecha', ascending=False).head(1)
#Filtrar 
consejeria

,busqueda,usuario,usuario_id,texto,fecha,url,link_gaceta
0,gaceta oficial cdmx,ConsejeríaCDMX,1928683574,Consulta la Gaceta Oficial del 17 de Junio de ...,2025-06-17 14:44:28+00:00,"[{'display_url': 'shre.ink/x6xM', 'expanded_ur...",https://shre.ink/x6xM


In [12]:
# async def login_and_get_user_tweets(user_id='1928683574', max_retries=3, retry_delay=30):
#     await client.login(
#         auth_info_1=USERNAME,
#         auth_info_2=EMAIL,
#         password=PASSWORD
#     )

#     all_data = []
#     retries = 0

#     while retries <= max_retries:
#         try:
#             tweets = await client.get_user_tweets(user_id, 'Tweets')

#             for tweet in tweets:
#                 all_data.append({
#                     "usuario": tweet.user.name,
#                     "usuario_id": tweet.user.id,
#                     "texto": tweet.text,
#                     "fecha": tweet.created_at,
#                     "url": tweet.urls
#                 })

#             break  # salir del ciclo si fue exitoso

#         except Exception as e:
#             print(f"⚠️ Error al obtener tuits del usuario '{user_id}': {e}")
#             if "rate limit" in str(e).lower():
#                 retries += 1
#                 print(f"🔁 Reintentando en {retry_delay} segundos... (Intento {retries}/{max_retries})")
#                 await asyncio.sleep(retry_delay)
#             else:
#                 break  # otro tipo de error, salir

#     df = pd.DataFrame(all_data)
#     return df


In [13]:
# df = await login_and_get_user_tweets('1928683574')
# df['fecha'] = pd.to_datetime(df['fecha'], format="%a %b %d %H:%M:%S %z %Y", errors='coerce')
# df

In [14]:
# consejeria = df[df['texto'].str.contains("Consulta la Gaceta Oficial", case=False, na=False)].copy()
# consejeria['link_gaceta'] = consejeria['url'].apply(lambda x: x[0]['expanded_url'] if isinstance(x, list) and len(x) > 0 and 'expanded_url' in x[0] else None)
# consejeria

In [15]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'
}

In [16]:
# Configurar Selenium sin abrir la ventana
options = Options()
options.add_argument("--headless")  
driver = webdriver.Chrome(options=options)

os.makedirs("gacetas_descargadas", exist_ok=True)

for i, row in consejeria.iterrows():
    url = f"{row['link_gaceta']}"
    nombre_archivo = f"gaceta_{row['fecha'].date()}.pdf"
    ruta_destino = os.path.join("gacetas_descargadas", nombre_archivo)

    try:
        print(f"🌐 Navegando a {url}")
        driver.get(url)
        time.sleep(5)  # espera a que redireccione

        final_url = driver.current_url
        print(f"🔗 Redirigido a: {final_url}")

        # Descargar con requests
        r = requests.get(final_url)
        if r.status_code == 200:
            with open(ruta_destino, 'wb') as f:
                f.write(r.content)
            print(f"✅ Guardado como: {ruta_destino}")
        else:
            print(f"❌ Error {r.status_code} al descargar {final_url}")

    except Exception as e:
        print(f"⚠️ Error con {url}: {e}")

driver.quit()


🌐 Navegando a https://shre.ink/x6xM
🔗 Redirigido a: https://data.consejeria.cdmx.gob.mx/portal_old/uploads/gacetas/2a7d8b2b3b8031ec5107fad4b770f621.pdf
✅ Guardado como: gacetas_descargadas\gaceta_2025-06-17.pdf


In [17]:
# Carpeta con archivos
carpeta = "gacetas_descargadas"

# Listar archivos PDF y ordenarlos por nombre (asumiendo formato: gaceta_YYYY-MM-DD.pdf)
archivos = sorted(
    [f for f in os.listdir(carpeta) if f.endswith(".pdf")],
    reverse=True
)

# Seleccionar el más reciente
ruta_pdf = os.path.join(carpeta, archivos[0])

print(f"📄 Archivo PDF más reciente: {ruta_pdf}")

📄 Archivo PDF más reciente: gacetas_descargadas\gaceta_2025-06-17.pdf


In [20]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
palabras_clave = ["Jefatura",
    "Secretaría", "Consejería", "Alcaldía", "Instituto", "Fiscalía",
    "Corporación", "Tribunal", "Procuraduría", "Comisión", "Agencia",
    "Servicio","Sistema","Agencia","Congreso","Fideicomiso", "Centro"
]

rotulos_a_eliminar = [
    "A L C A L D Í A S",
    "O R G A N I S M O S   A U T Ó N O M O S",
    "C O N V O C A T O R I A S   D E   L I C I T A C I Ó N   Y   F A L L O S",
    "E M P R E S A S   D E   P A R T I C I P A C I Ó N   E S T A T A L",
    "Í  N  D  I  C  E P O D E R   E J E C U T I V O"
]

def unir_bloques_por_linea(blocks, tolerancia=2.5):
    texto_unido = []
    bloque_actual = ""
    y_anterior = None
    for block in blocks:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            y_actual = line["bbox"][1]
            texto_linea = " ".join([span["text"] for span in line["spans"]]).strip()
            if y_anterior is None or abs(y_actual - y_anterior) <= tolerancia:
                bloque_actual += " " + texto_linea
            else:
                if bloque_actual:
                    texto_unido.append(bloque_actual.strip())
                bloque_actual = texto_linea
            y_anterior = y_actual
    if bloque_actual:
        texto_unido.append(bloque_actual.strip())
    return texto_unido

# Abrir PDF
doc = fitz.open(ruta_pdf)
lineas_unidas = []
for page_num in range(min(4, len(doc))):
    page = doc[page_num]
    blocks = page.get_text("dict")["blocks"]
    lineas_unidas += unir_bloques_por_linea(blocks)

# Extraer líneas útiles y fecha
contenido_util = []
fecha_documento = ""
for linea in lineas_unidas:
    linea = linea.strip()
    if not fecha_documento:
        match = re.search(r"\d{1,2} de [a-z]+ de \d{4}", linea.lower())
        if match:
            fecha_documento = match.group(0)

    if any(frag in linea for frag in [
        "Órgano de Difusión del Gobierno de la Ciudad de México",
        "Índice", "Continúa en la Pág.",
        "Viene de la Pág.",
        "GACETA OFICIAL DE LA CIUDAD DE MÉXICO",
        "Este ejemplar se acompaña de un anexo electrónico",
        "Época", "No."
    ]):
        continue

    if "E D I C T O S" in linea.upper():
        break

    contenido_util.append(linea)

# Procesar con retroceso a organismo más reciente
organismos = []
textos = []

registro_actual = ""
organismo_actual = None

for i, linea in enumerate(contenido_util):
    # Si contiene registro
    if "\uf0a8" in linea:
        # Guardar texto previo si hay
        if registro_actual.strip() and organismo_actual:
            for r in rotulos_a_eliminar:
                registro_actual = registro_actual.replace(r, "")
            registro_actual = re.sub(r"(?:[A-ZÁÉÍÓÚÑ] ?){6,}", "", registro_actual).strip()
            if registro_actual:
                textos.append(registro_actual)
                organismos.append(organismo_actual)

        registro_actual = linea.replace("\uf0a8", "").strip()

        # Buscar el organismo más cercano hacia atrás
        for j in range(i - 1, -1, -1):
            if any(contenido_util[j].startswith(p) for p in palabras_clave):
                organismo_actual = contenido_util[j]
                break
    else:
        registro_actual += " " + linea

# Último registro
if registro_actual and organismo_actual:
    for r in rotulos_a_eliminar:
        registro_actual = registro_actual.replace(r, "")
    registro_actual = re.sub(r"(?:[A-ZÁÉÍÓÚÑ] ?){6,}", "", registro_actual).strip()
    if registro_actual:
        textos.append(registro_actual)
        organismos.append(organismo_actual)
# Convertir fecha_documento (ej. "10 de junio de 2025") a datetime
fecha_dt = datetime.strptime(fecha_documento, "%d de %B de %Y")

# Crear DataFrame
df = pd.DataFrame({
    "fecha": [fecha_documento] * len(textos),
    "organismo": organismos,
    "texto": textos
})


#Añadir link de la gaceta que viene en final_url
df['link_gaceta'] = final_url

df['fecha'] = pd.to_datetime(df['fecha'], format="%d de %B de %Y", errors='coerce')

# Extraer solo fecha sin hora
df['fecha'] = df['fecha'].dt.date
# Construir nombre de archivo
fecha_str = df['fecha'].astype(str).iloc[0]
df.to_excel(f"gacetas_descargadas/resultados_gacetas/gaceta_{fecha_str}.xlsx", index=False)
df

,fecha,organismo,texto,link_gaceta
0,2025-06-17,Secretaría del Medio Ambiente de la Ciudad de ...,Nota aclaratoria al Aviso por el cual se da a ...,https://data.consejeria.cdmx.gob.mx/portal_old...
1,2025-06-17,Secretaría del Medio Ambiente de la Ciudad de ...,Nota aclaratoria al Aviso por el cual se da a ...,https://data.consejeria.cdmx.gob.mx/portal_old...
2,2025-06-17,"Secretaría de Planeación, Ordenamiento Territo...",Resolución para la solicitud de cambio de uso ...,https://data.consejeria.cdmx.gob.mx/portal_old...
3,2025-06-17,Secretaría de Pueblos y Barrios Originarios y ...,Aviso por el que se da a conocer la Convocator...,https://data.consejeria.cdmx.gob.mx/portal_old...
4,2025-06-17,Alcaldía Azcapotzalco,Aviso por el que se dan a conocer las Reglas d...,https://data.consejeria.cdmx.gob.mx/portal_old...
5,2025-06-17,Secretaría de Cultura de la Ciudad de México,Licitación pública nacional número /LPN/05/202...,https://data.consejeria.cdmx.gob.mx/portal_old...
6,2025-06-17,Secretaría de Turismo de la Ciudad de México,Licitación pública nacional número -LPN-21-202...,https://data.consejeria.cdmx.gob.mx/portal_old...
7,2025-06-17,Secretaría de Turismo de la Ciudad de México,Licitación pública nacional número /LPN/009/20...,https://data.consejeria.cdmx.gob.mx/portal_old...
